In [31]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [32]:
# Fungsi untuk mengambil data historis harga saham dari Yahoo Finance
def get_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

In [33]:
# Tanggal mulai dan akhir yang diinginkan
start_date = "2019-01-20"
end_date = "2023-06-15"

In [34]:
# Simbol saham Bank Rakyat Indonesia di Yahoo Finance
stock_symbol = "BBRI.JK"

In [35]:
# Mengambil data historis harga saham
df = get_stock_data(stock_symbol, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [37]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-21,3800.0,3830.0,3800.0,3800.0,3149.416992,80926000
2019-01-22,3770.0,3790.0,3740.0,3770.0,3124.553467,119687600
2019-01-23,3760.0,3810.0,3730.0,3770.0,3124.553467,163458700
2019-01-24,3750.0,3810.0,3750.0,3790.0,3141.129395,176166500
2019-01-25,3820.0,3820.0,3780.0,3780.0,3132.841309,127141300


In [36]:
# Mengecek nilai null pada kolom 'Close'
null_values = df['Close'].isnull().sum()

# Output jumlah nilai null
print("Jumlah nilai null dalam kolom 'Close':", null_values)

KeyError: "None of ['Date'] are in the columns"

In [ ]:
# Menghapus baris yang memiliki nilai NaN di kolom 'Close'
df.dropna(subset=['Close'], inplace=True)

In [ ]:
null_values = df['Close'].isnull().sum()
print("Jumlah nilai null dalam kolom 'Close':", null_values)  # Harus 0

In [ ]:
# Mengambil kolom 'Close' sebagai data harga saham yang akan diprediksi
data = df['Close'].values.reshape(-1, 1)

# Normalisasi data ke dalam rentang 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

In [ ]:
print(data_scaled )

In [ ]:
# Membagi data menjadi data training dan data testing
train_size = int(len(data_scaled) * 0.8)  # 80% data untuk training
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

print("Jumlah data pelatihan:", len(train_data))
print("Jumlah data pengujian:", len(test_data))


In [ ]:
# Membuat dataset yang sesuai dengan model LSTM
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 30  # Jumlah data historis yang digunakan untuk memprediksi data berikutnya
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

# Reshape data ke bentuk [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Step 2: Membangun model LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Melatih model
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

In [ ]:
# Step 3: Prediksi data testing
y_pred = model.predict(X_test)

# Mengubah data kembali ke skala aslinya
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Step 4: Visualisasi hasil prediksi
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Data Asli')
plt.plot(y_pred, label='Prediksi')
plt.title('Prediksi Harga Saham menggunakan LSTM')
plt.xlabel('Tanggal')
plt.ylabel('Harga Saham')
plt.legend()
plt.show()